# YOLO V1
[You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640.pdf)
## 贡献
Yolov1是第一个One-stage的目标检测算法，将Two-stage的分类和回归问题统一为了一个回归问题。

**核心思想**：将整张图片作为网络输入，直接在输出层对Bounding Box的位置和类别进行回归。

**实现方法**：将图片分为$S \times S$个栅格(grid)，如果某个Object的中心点落在了这个栅格中，就由这个栅格负责预测这个Object。每个栅格需要预测$B$个BoundingBox的位置信息$(x,y,h,w)$和置信度$(c)$，此外还要预测这个Object是每个类别的条件概率。

* **(x,y,h,w)**: 分别代表的是Bounding Box的中心位置和宽高

* **confidence**: 代表了所预测的box中是否存在Object，以及该Box预测准确度。公式如下：
    $$ confidence = P_r(Object)\times IOU_{pred}^{truth}$$
    $P_r(Object)$表示Object是否落在这个grid里，如果grid里有Object,$P_r(Object) = 1$，否则，$P_r(Object) = 0$；
    
    $IOU$表示的是该BoundingBox的准确度,$pred$是预测出的BBox的区域位置，$truth$是标签区域位置，$IOU$就是求两个区域的交集比并集

<div align = "center", style="height:20%;width:20%"><img src = "./img/IOU.png"></img></div>

* **条件概率**


## 算法步骤
### 1. 神经网络提取网格
<div align = "center", style="height:50%;width:50%"><img src = "./img/yolov1_network.png"></img></div>




### 2. NMS(非极大值抑制)

In [1]:
# YOLO V1